In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('appartments.csv').drop(22)

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Palam Vihar', 'Park Hospital', 'Gurgaon Railway Station']","{'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}",https://www.99acres.com/smartworld-one-dxp-sector-113-gurgaon-npxid-r400415,"{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,370 sq.ft.', 'price-range': '₹ 2 - 2.4 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,850 - 2,050 sq.ft.', 'price-range': '₹ 2.25 - 3.59 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,600 sq.ft.', 'price-range': '₹ 3.24 - 4.56 Cr'}}","['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap University', 'Park Hospital, Palam Vihar', 'Pacific D21 Mall', 'Palam Vihar Halt Railway Station']","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The NorthCap University': '4.4 Km', 'Park Hospital, Palam Vihar': '1.4 Km', 'Pacific D21 Mall': '8.2 Km', 'Palam Vihar Halt Railway Station': '1.2 Km', 'Dwarka Sector 21 Metro Station': '8.1 Km', 'Dwarka Expressway': '450 m', 'Fun N Food Water Park': '8.1 Km', 'Indira Gandhi International Airport': '14.1 Km', 'Tau DeviLal Sports Complex': '11.2 Km', 'Hamoni Golf Camp': '5 Km', 'Hyatt Place': '6.1 Km', 'Altrade Business Centre': '11.2 Km'}",https://www.99acres.com/m3m-crown-sector-111-gurgaon-npxid-r404068,"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}","['Bowling Alley', 'Mini Theatre', 'Manicured Garden', 'Swimming Pool', 'Flower Garden', 'Reading Lounge', 'Golf Course', 'Barbecue', 'Sauna']"
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63, Gurgaon","['AIPL Business Club Sector 62', 'Heritage Xperiential Learning School', 'CK Birla Hospital', 'Paras Trinity Mall Sector 63', 'Rapid Metro Station Sector 56']","{'AIPL Business Club Sector 62': '2.7 Km', 'Heritage Xperiential Learning School': '2 Km', 'CK Birla Hospital': '2.5 Km', 'Paras Trinity Mall Sector 63': '3.5 Km', 'Rapid Metro Station Sector 56': '3.8 Km', 'De Adventure Park': '6.8 Km', 'Golf Course Ext Rd': '99 Meter', 'DoubleTree by Hilton Hotel Gurgaon': '3.6 Km', 'KIIT College of Engineering Sohna Road': '8.4 Km', 'Mehrauli-Gurgaon Road': '11.8 Km', 'Indira Gandhi International Airport': '21.1 Km', 'Nirvana Rd': '160 Meter', 'TERI Golf Course': '8.7 Km'}",https://www.99acres.com/adani-brahma-samsara-vilasa-sector-63-gurgaon-npxid-r327995,"{'3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Super Built-up Area', 'area': '1,800 - 3,150 sq.ft.', 'price-range': '₹ 2.43 - 15.75 Cr'}, '4 BHK': {'building_type': 'Independent Floor', 'area_type': 'Super Built-up Area', 'area': '2,750 - 4,500 sq.ft.', 'price-range': '₹ 3.36 - 22.5 Cr'}, 'Land': {'building_type': '', 'area_type': 'Plot Area', 'area': '500 - 4,329 sq.ft.', 'price-range': '₹ 2.05 - 41.13 Cr'}}","['Terrace Garden', 'Gazebo', 'Fountain', 'Amphitheatre', 'Party Lawn', 'Basketball Court', 'Badminton Court', 'Yoga/Meditation Area', 'Indoor Games']"
3,Sobha City,"2, 3, 4 BHK Apartment in S

In [6]:
df[['PropertyName', 'TopFacilities']]

,PropertyName,TopFacilities
0,Smartworld One DXP,"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"
1,M3M Crown,"['Bowling Alley', 'Mini Theatre', 'Manicured Garden', 'Swimming Pool', 'Flower Garden', 'Reading Lounge', 'Golf Course', 'Barbecue', 'Sauna']"
2,Adani Brahma Samsara Vilasa,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amphitheatre', 'Party Lawn', 'Basketball Court', 'Badminton Court', 'Yoga/Meditation Area', 'Indoor Games']"
3,Sobha City,"['Swimming Pool', 'Volley Ball Court', 'Aerobics Centre', 'Card Room', 'Barbecue', 'Sauna', 'Steam Room', 'Creche/Day care', 'Skating Rink']"
4,Signature Global City 93,"['Mini Theatre', 'Doctor on Call', 'Concierge Service', 'Swimming Pool', 'Bar/Chill-Out Lounge', 'Laundry', 'Flower Garden', 'Reflexology Park', 'Salon']"
...,...,...
242,DLF Princeton Estate,"['Swimming Pool', 'Medical Centre', 'Laundry', 'Salon', 'Grocery Shop', 'Property Staff', 'Shopping Centre', 'Table Tennis', 'Badminton Court']"
243,Pyramid Urban Homes 2,"['Shopping Centre', 'Community Hall', '24x7 Security', 'Gated Community', 'Landscape Garden', 'Lift(s)', 'Car Parking']"
244,Satya The Hermitage,"['Bus Shelter', 'Swimming Pool', 'Business Lounge', 'Reading Lounge', 'Barbecue', 'Skating Rink', 'Fountain', 'Lawn Tennis Court', 'Amphitheatre']"
245,BPTP Spacio,"['Swimming Pool', 'Card Room', 'Piped Gas', 'Pool Table', 'Sun Deck', 'Lawn Tennis Court', 'Multipurpose Court', 'Earthquake Resistant', 'Toddler Pool']"


In [7]:
def extract_list(s) :
  return re.findall(r"'(.*?)'", s)

In [8]:
df['TopFacilities'].loc[0]  # list within a string

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

In [9]:
extract_list(df['TopFacilities'].loc[0]) # simple list

['Swimming Pool',
 'Salon',
 'Restaurant',
 'Spa',
 'Cafeteria',
 'Sun Deck',
 '24x7 Security',
 'Club House',
 'Gated Community']

In [10]:
df['TopFacilities']= df['TopFacilities'].apply(extract_list)

In [11]:
df['FacilitiesStr'] = df['TopFacilities'].apply(' '.join) # list to string

In [12]:
df['FacilitiesStr'].sample()

,FacilitiesStr
27,Swimming Pool Reflexology Park Jacuzzi Skating Rink Cricket Pitch Multipurpose Court Amphitheatre Toddler Pool Basketball Court


In [13]:
# convert string to vector
tdidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

In [14]:
tfidf_matrix = tdidf_vectorizer.fit_transform(df['FacilitiesStr'])

In [15]:
tfidf_matrix.toarray().shape # collection of 247 vectors in 953 dimension space

(246, 953)

In [16]:
cosine_sim1 = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [17]:
cosine_sim1.shape # distance of a vector with all the other vectors

(246, 246)

In [18]:
def recommed_properties(property_name, cosine_sim=cosine_sim1) :
  # find index of the property
  idx = int(df[df['PropertyName'] == property_name].index[0])
  # add index(numbering) to properties
  sim_scores = list(enumerate(cosine_sim[idx]))
  # sort based on distance in descending
  sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
  # top 5 exlucding the first one (vi with vi)
  sim_scores = sim_scores[1:6]
  # extract indices
  property_indices = [i[0] for i in sim_scores]
  recommendations_df = pd.DataFrame({
      'PropertyName' : df['PropertyName'].iloc[property_indices],
      'SimilarityScore' : sim_scores
  })
  return recommendations_df


In [19]:
recommed_properties('DLF The Arbour') # based on top facilities

,PropertyName,SimilarityScore
64,Ace Palm Floors,"(63, 0.4529382062441955)"
217,Yashika 104,"(216, 0.4199606322926784)"
93,JMS The Nation,"(92, 0.4166584649363288)"
154,India Rashtra,"(153, 0.398954234680194)"
0,Smartworld One DXP,"(0, 0.38885046199432893)"


In [20]:
df[['PropertyName', 'PriceDetails']].loc[0]

,0
PropertyName,Smartworld One DXP
PriceDetails,"{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,370 sq.ft.', 'price-range': '₹ 2 - 2.4 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,850 - 2,050 sq.ft.', 'price-range': '₹ 2.25 - 3.59 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,600 sq.ft.', 'price-range': '₹ 3.24 - 4.56 Cr'}}"


In [21]:
text = df['PriceDetails'].loc[0]
text

"{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,370 sq.ft.', 'price-range': '₹ 2 - 2.4 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,850 - 2,050 sq.ft.', 'price-range': '₹ 2.25 - 3.59 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,600 sq.ft.', 'price-range': '₹ 3.24 - 4.56 Cr'}}"

In [22]:
import ast

parsed = ast.literal_eval(text)
parsed

{'2 BHK': {'building_type': 'Apartment',
  'area_type': 'Carpet Area',
  'area': '1,370 sq.ft.',
  'price-range': '₹ 2 - 2.4 Cr'},
 '3 BHK': {'building_type': 'Apartment',
  'area_type': 'Carpet Area',
  'area': '1,850 - 2,050 sq.ft.',
  'price-range': '₹ 2.25 - 3.59 Cr'},
 '4 BHK': {'building_type': 'Apartment',
  'area_type': 'Carpet Area',
  'area': '2,600 sq.ft.',
  'price-range': '₹ 3.24 - 4.56 Cr'}}

In [23]:
type(parsed)

dict

In [24]:
print(parsed.keys())

dict_keys(['2 BHK', '3 BHK', '4 BHK'])


In [25]:
parsed['2 BHK'].keys()

dict_keys(['building_type', 'area_type', 'area', 'price-range'])

In [26]:
df.isnull().sum()

,0
PropertyName,0
PropertySubName,0
NearbyLocations,0
LocationAdvantages,0
Link,0
PriceDetails,0
TopFacilities,0
FacilitiesStr,0


In [27]:
list_of_values = []

In [28]:
st = set()

In [29]:
def create_feature (parsed):
  d  = {}
  pattern = re.compile(r'(₹|sq\.ft\.|Cr|,|L)')
  for key in parsed.keys():
    for key2, value in parsed[key].items():
      if key2 == 'area_type' :
        continue
      if 'Price on Request' in value:
        d[key2+'_'+key+'_'+'low'] = None
        df[key2 +'_'+key+'_'+'high'] = None
      elif ('sq.ft.' in value) | ('Cr' in value) | ('₹' in value) | ('L' in value):
        if len(value.split('-')) == 1 :
          value = re.sub(pattern, '', value)
          d[key2+'_'+key+'_'+ 'low'] = value.strip()
          d[key2+'_'+key+'_'+'high'] = value.strip()
        else :
          low, high = value.split('-')
          low = re.sub(pattern, '', low)
          high = re.sub(pattern, '', high)
          d[key2+'_'+key+'_'+ 'low'] = low.strip()
          d[key2+'_'+key+'_'+'high'] = high.strip()
      else :
        d[key2+'_'+key] = value.strip()
  list_of_values.append(d)
  st.update(d.keys())

In [30]:
for row in df['PriceDetails']:
  try:
    parsed = ast.literal_eval(row)
    create_feature(parsed)
  except:
    create_feature({})

In [31]:
for i in sorted(st):
  print(i)

area_1 BHK_high
area_1 BHK_low
area_1 RK_high
area_1 RK_low
area_2 BHK_high
area_2 BHK_low
area_3 BHK_high
area_3 BHK_low
area_4 BHK_high
area_4 BHK_low
area_5 BHK_high
area_5 BHK_low
area_6 BHK_high
area_6 BHK_low
area_Land_high
area_Land_low
building_type_1 BHK
building_type_1 RK
building_type_2 BHK
building_type_3 BHK
building_type_4 BHK
building_type_5 BHK
building_type_6 BHK
building_type_Land
price-range_1 BHK_high
price-range_1 BHK_low
price-range_1 RK_high
price-range_1 RK_low
price-range_2 BHK_high
price-range_2 BHK_low
price-range_3 BHK_high
price-range_3 BHK_low
price-range_4 BHK_high
price-range_4 BHK_low
price-range_5 BHK_high
price-range_5 BHK_low
price-range_6 BHK_high
price-range_6 BHK_low
price-range_Land_high
price-range_Land_low


In [32]:
len(st)

40

In [33]:
feature_set = st

In [34]:
def sort_key(item):
    # Extract BHK type (1, 2, ..., Land, RK)
    match = re.search(r"(1 RK|1 BHK|2 BHK|3 BHK|4 BHK|5 BHK|6 BHK|Land)", item)
    bhk_type = match.group(1) if match else ""

    # Give numeric value to each BHK type for sorting
    bhk_order = {
        "1 RK": 0,
        "1 BHK": 1,
        "2 BHK": 2,
        "3 BHK": 3,
        "4 BHK": 4,
        "5 BHK": 5,
        "6 BHK": 6,
        "Land": 7
    }

    # Return tuple for sorting: (bhk_type_sort_value, original item)
    return (bhk_order.get(bhk_type, 99), item)


In [35]:
feature_set = sorted(list(feature_set), key=sort_key)

In [36]:
def extract_values (index, df):
  values = list_of_values[index]
  helper = []
  for key in feature_set :
    if key in values.keys():
      helper.append(values[key])
    else :
      helper.append(None)
  df_helper.loc[index] = helper

In [37]:
print(len(list_of_values))

246


In [38]:
for row in df['PriceDetails'].index:
  if df.loc[row, 'PriceDetails'] == 'PriceDetails':
    print(row)

In [39]:
df_helper = pd.DataFrame(columns=list(feature_set))

In [40]:
for i in range(len(list_of_values)):
  extract_values(i, df_helper)

In [41]:
pd.set_option('display.max_columns', None)

In [42]:
df_helper.sample(5)

,area_1 RK_high,area_1 RK_low,building_type_1 RK,price-range_1 RK_high,price-range_1 RK_low,area_1 BHK_high,area_1 BHK_low,building_type_1 BHK,price-range_1 BHK_high,price-range_1 BHK_low,area_2 BHK_high,area_2 BHK_low,building_type_2 BHK,price-range_2 BHK_high,price-range_2 BHK_low,area_3 BHK_high,area_3 BHK_low,building_type_3 BHK,price-range_3 BHK_high,price-range_3 BHK_low,area_4 BHK_high,area_4 BHK_low,building_type_4 BHK,price-range_4 BHK_high,price-range_4 BHK_low,area_5 BHK_high,area_5 BHK_low,building_type_5 BHK,price-range_5 BHK_high,price-range_5 BHK_low,area_6 BHK_high,area_6 BHK_low,building_type_6 BHK,price-range_6 BHK_high,price-range_6 BHK_low,area_Land_high,area_Land_low,building_type_Land,price-range_Land_high,price-range_Land_low
244,None,None,None,None,None,None,None,None,None,None,1079,1000,Apartment,None,None,1865,1225,Apartment,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
80,None,None,None,None,None,None,None,None,None,None,1719,1565,Apartment,1.72,1.49,2475,2150,Apartment,2.48,2.04,2925,2925,Apartment,2.93,2.78,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
28,None,None,None,None,None,None,None,None,None,None,1105,1105,Independent Floor,93.93,88,1423,1423,Independent Floor,1.13,1.03,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
186,None,None,None,None,None,None,None,None,None,None,1385,1385,Apartment,1.09,1.09,1900,1665,Apartment,1.51,1.32,4150,2355,Apartment,3.31,1.87,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
55,None,None,None,None,None,None,None,None,None,None,2370,2246,Apartment,8.42,7.8,3518,2651,Apartment,12.34,9.3,4969,3081,Apartment,15.32,9.5,6147,6147,Apartment,18.44,18.44,None,None,None,None,None,None,None,None,None,None


In [43]:
df.loc[11]['PriceDetails']

"{'4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '3,950 sq.ft.', 'price-range': '₹ 7 - 7.31 Cr'}}"

In [44]:
df_helper.index = df['PropertyName']

In [45]:
df_helper.sample(5)

,area_1 RK_high,area_1 RK_low,building_type_1 RK,price-range_1 RK_high,price-range_1 RK_low,area_1 BHK_high,area_1 BHK_low,building_type_1 BHK,price-range_1 BHK_high,price-range_1 BHK_low,area_2 BHK_high,area_2 BHK_low,building_type_2 BHK,price-range_2 BHK_high,price-range_2 BHK_low,area_3 BHK_high,area_3 BHK_low,building_type_3 BHK,price-range_3 BHK_high,price-range_3 BHK_low,area_4 BHK_high,area_4 BHK_low,building_type_4 BHK,price-range_4 BHK_high,price-range_4 BHK_low,area_5 BHK_high,area_5 BHK_low,building_type_5 BHK,price-range_5 BHK_high,price-range_5 BHK_low,area_6 BHK_high,area_6 BHK_low,building_type_6 BHK,price-range_6 BHK_high,price-range_6 BHK_low,area_Land_high,area_Land_low,building_type_Land,price-range_Land_high,price-range_Land_low
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Shree Vardhman City,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1305,1125,,1.31,1.13
M3M Golf Hills,None,None,None,None,None,None,None,None,None,None,1420,1420,Apartment,1.5,1.49,2155,1532,Apartment,3.49,1.61,2677,2630,Apartment,4.34,2.78,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Trump Tower,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3525,3525,Apartment,8.64,8.2,6050,4550,Apartment,15.29,10.3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Ansals Shiva Som Valley,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1611,1161,,1.29,93
Emaar Palm Gardens,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1950,1720,Apartment,1.54,1.36,None,None,None,None,None,3750,3750,Apartment,3.28,3.28,None,None,None,None,None,None,None,None,None,None


In [46]:
df_helper.shape

(246, 40)

In [47]:
categorical_cols = df_helper.select_dtypes(include=['object']).columns
categorical_cols

Index(['area_1 RK_high', 'area_1 RK_low', 'building_type_1 RK',
       'price-range_1 RK_high', 'price-range_1 RK_low', 'area_1 BHK_high',
       'area_1 BHK_low', 'building_type_1 BHK', 'price-range_1 BHK_high',
       'price-range_1 BHK_low', 'area_2 BHK_high', 'area_2 BHK_low',
       'building_type_2 BHK', 'price-range_2 BHK_high',
       'price-range_2 BHK_low', 'area_3 BHK_high', 'area_3 BHK_low',
       'building_type_3 BHK', 'price-range_3 BHK_high',
       'price-range_3 BHK_low', 'area_4 BHK_high', 'area_4 BHK_low',
       'building_type_4 BHK', 'price-range_4 BHK_high',
       'price-range_4 BHK_low', 'area_5 BHK_high', 'area_5 BHK_low',
       'building_type_5 BHK', 'price-range_5 BHK_high',
       'price-range_5 BHK_low', 'area_6 BHK_high', 'area_6 BHK_low',
       'building_type_6 BHK', 'price-range_6 BHK_high',
       'price-range_6 BHK_low', 'area_Land_high', 'area_Land_low',
       'building_type_Land', 'price-range_Land_high', 'price-range_Land_low'],
      dtype='obj

In [48]:
df_helper['area_1 BHK_high'] = df_helper['area_1 BHK_high'].astype(float)

In [49]:
df_helper['area_1 RK_low'] = df_helper['area_1 RK_low'].astype(float)

In [50]:
df_helper['area_1 BHK_low']= df_helper['area_1 BHK_low'].astype(float)

In [51]:
df_helper['area_1 RK_high'] = df_helper['area_1 RK_high'].astype(float)
df_helper['area_2 BHK_low'] = df_helper['area_2 BHK_low'].astype(float)
df_helper['area_2 BHK_high'] = df_helper['area_2 BHK_high'].astype(float)
df_helper['area_3 BHK_low'] = df_helper['area_3 BHK_low'].astype(float)
df_helper['area_3 BHK_high'] = df_helper['area_3 BHK_high'].astype(float)
df_helper['area_4 BHK_low'] = df_helper['area_4 BHK_low'].astype(float)
df_helper['area_4 BHK_high'] = df_helper['area_4 BHK_high'].astype(float)
df_helper['area_5 BHK_low'] = df_helper['area_5 BHK_low'].astype(float)
df_helper['area_5 BHK_high'] = df_helper['area_5 BHK_high'].astype(float)
df_helper['area_6 BHK_low'] = df_helper['area_6 BHK_low'].astype(float)
df_helper['area_6 BHK_high'] = df_helper['area_6 BHK_high'].astype(float)
df_helper['area_Land_low'] = df_helper['area_Land_low'].astype(float)
df_helper['area_Land_high'] = df_helper['area_Land_high'].astype(float)

In [52]:
df_helper['price-range_1 BHK_high'] = df_helper['price-range_1 BHK_high'].astype(float)
df_helper['price-range_1 BHK_low'] = df_helper['price-range_1 BHK_low'].astype(float)
df_helper['price-range_1 RK_high'] = df_helper['price-range_1 RK_high'].astype(float)
df_helper['price-range_1 RK_low'] = df_helper['price-range_1 RK_low'].astype(float)
df_helper['price-range_2 BHK_high'] = df_helper['price-range_2 BHK_high'].astype(float)
df_helper['price-range_2 BHK_low'] = df_helper['price-range_2 BHK_low'].astype(float)
df_helper['price-range_3 BHK_high'] = df_helper['price-range_3 BHK_high'].astype(float)
df_helper['price-range_3 BHK_low'] = df_helper['price-range_3 BHK_low'].astype(float)
df_helper['price-range_4 BHK_high'] = df_helper['price-range_4 BHK_high'].astype(float)
df_helper['price-range_4 BHK_low'] = df_helper['price-range_4 BHK_low'].astype(float)
df_helper['price-range_5 BHK_high'] = df_helper['price-range_5 BHK_high'].astype(float)
df_helper['price-range_5 BHK_low'] = df_helper['price-range_5 BHK_low'].astype(float)
df_helper['price-range_6 BHK_high'] = df_helper['price-range_6 BHK_high'].astype(float)
df_helper['price-range_6 BHK_low'] = df_helper['price-range_6 BHK_low'].astype(float)
df_helper['price-range_Land_high'] = df_helper['price-range_Land_high'].astype(float)
df_helper['price-range_Land_low'] = df_helper['price-range_Land_low'].astype(float)

In [53]:
categorical_cols = df_helper.select_dtypes(include=['object']).columns
categorical_cols

Index(['building_type_1 RK', 'building_type_1 BHK', 'building_type_2 BHK',
       'building_type_3 BHK', 'building_type_4 BHK', 'building_type_5 BHK',
       'building_type_6 BHK', 'building_type_Land'],
      dtype='object')

In [54]:
df_helper.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, Smartworld One DXP to SS The Coralwood
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   area_1 RK_high          3 non-null      float64
 1   area_1 RK_low           3 non-null      float64
 2   building_type_1 RK      3 non-null      object 
 3   price-range_1 RK_high   2 non-null      float64
 4   price-range_1 RK_low    2 non-null      float64
 5   area_1 BHK_high         18 non-null     float64
 6   area_1 BHK_low          18 non-null     float64
 7   building_type_1 BHK     18 non-null     object 
 8   price-range_1 BHK_high  8 non-null      float64
 9   price-range_1 BHK_low   8 non-null      float64
 10  area_2 BHK_high         121 non-null    float64
 11  area_2 BHK_low          121 non-null    float64
 12  building_type_2 BHK     121 non-null    object 
 13  price-range_2 BHK_high  95 non-null     float64
 14  price-range_2 BHK

In [55]:
ohe_df = pd.get_dummies(df_helper, columns=categorical_cols, drop_first=True)

In [56]:
ohe_df.fillna(0, inplace=True)

In [57]:
ohe_df.shape

(246, 43)

In [58]:
ohe_df.columns

Index(['area_1 RK_high', 'area_1 RK_low', 'price-range_1 RK_high',
       'price-range_1 RK_low', 'area_1 BHK_high', 'area_1 BHK_low',
       'price-range_1 BHK_high', 'price-range_1 BHK_low', 'area_2 BHK_high',
       'area_2 BHK_low', 'price-range_2 BHK_high', 'price-range_2 BHK_low',
       'area_3 BHK_high', 'area_3 BHK_low', 'price-range_3 BHK_high',
       'price-range_3 BHK_low', 'area_4 BHK_high', 'area_4 BHK_low',
       'price-range_4 BHK_high', 'price-range_4 BHK_low', 'area_5 BHK_high',
       'area_5 BHK_low', 'price-range_5 BHK_high', 'price-range_5 BHK_low',
       'area_6 BHK_high', 'area_6 BHK_low', 'price-range_6 BHK_high',
       'price-range_6 BHK_low', 'area_Land_high', 'area_Land_low',
       'price-range_Land_high', 'price-range_Land_low',
       'building_type_1 BHK_Service Apartment',
       'building_type_2 BHK_Independent Floor',
       'building_type_2 BHK_Service Apartment',
       'building_type_3 BHK_Independent Floor',
       'building_type_3 BHK_Service

In [59]:
ohe_df

,area_1 RK_high,area_1 RK_low,price-range_1 RK_high,price-range_1 RK_low,area_1 BHK_high,area_1 BHK_low,price-range_1 BHK_high,price-range_1 BHK_low,area_2 BHK_high,area_2 BHK_low,price-range_2 BHK_high,price-range_2 BHK_low,area_3 BHK_high,area_3 BHK_low,price-range_3 BHK_high,price-range_3 BHK_low,area_4 BHK_high,area_4 BHK_low,price-range_4 BHK_high,price-range_4 BHK_low,area_5 BHK_high,area_5 BHK_low,price-range_5 BHK_high,price-range_5 BHK_low,area_6 BHK_high,area_6 BHK_low,price-range_6 BHK_high,price-range_6 BHK_low,area_Land_high,area_Land_low,price-range_Land_high,price-range_Land_low,building_type_1 BHK_Service Apartment,building_type_2 BHK_Independent Floor,building_type_2 BHK_Service Apartment,building_type_3 BHK_Independent Floor,building_type_3 BHK_Service Apartment,building_type_3 BHK_Villa,building_type_4 BHK_Independent Floor,building_type_4 BHK_Villa,building_type_5 BHK_Independent Floor,building_type_5 BHK_Villa,building_type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1370.0,1370.0,2.40,2.00,2050.0,1850.0,3.59,2.25,2600.0,2600.0,4.56,3.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,False,False,False,False,False,False,False,False,False,False
M3M Crown,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,2170.0,1605.0,3.03,2.20,2670.0,2248.0,3.73,3.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,False,False,False,False,False,False,False,False,False,False
Adani Brahma Samsara Vilasa,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,3150.0,1800.0,15.75,2.43,4500.0,2750.0,22.50,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4329.0,500.0,41.13,2.05,False,False,False,True,False,False,True,False,False,False,False
Sobha City,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1692.0,1381.0,3.21,1.55,2343.0,1711.0,4.79,1.76,2963.0,2423.0,6.06,2.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,False,False,False,False,False,False,False,False,False,False
Signature Global City 93,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1118.0,981.0,1.06,93.01,1530.0,1235.0,1.45,1.12,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,964.0,964.0,1.35,1.35,1127.0,1127.0,1.55,1.55,1562.0,1562.0,2.15,2.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,False,False,False,False,False,False,False,False,False,False
Pyramid Urban Homes 2,0.0,0.0,0.0,0.0,398.0,335.0,27.86,23.45,625.0,500.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,False,False,False,False,False,False,False,False,False,False
Satya The Hermitage,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1450.0,1450.0,80.00,80.00,1991.0,1991.0,1.00,1.00,4711.0,2639.0,2.14,1.20,4731.0,4731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,False,False,False,False,False,False,False,False,False,False,False


In [60]:
ohe_df.isnull().sum()

,0
area_1 RK_high,0
area_1 RK_low,0
price-range_1 RK_high,0
price-range_1 RK_low,0
area_1 BHK_high,0
area_1 BHK_low,0
price-range_1 BHK_high,0
price-range_1 BHK_low,0
area_2 BHK_high,0
area_2 BHK_low,0


In [61]:
print(ohe_df.dtypes)

area_1 RK_high                           float64
area_1 RK_low                            float64
price-range_1 RK_high                    float64
price-range_1 RK_low                     float64
area_1 BHK_high                          float64
area_1 BHK_low                           float64
price-range_1 BHK_high                   float64
price-range_1 BHK_low                    float64
area_2 BHK_high                          float64
area_2 BHK_low                           float64
price-range_2 BHK_high                   float64
price-range_2 BHK_low                    float64
area_3 BHK_high                          float64
area_3 BHK_low                           float64
price-range_3 BHK_high                   float64
price-range_3 BHK_low                    float64
area_4 BHK_high                          float64
area_4 BHK_low                           float64
price-range_4 BHK_high                   float64
price-range_4 BHK_low                    float64
area_5 BHK_high     

In [62]:
ohe_df = ohe_df.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)

In [63]:
ohe_df

,area_1 RK_high,area_1 RK_low,price-range_1 RK_high,price-range_1 RK_low,area_1 BHK_high,area_1 BHK_low,price-range_1 BHK_high,price-range_1 BHK_low,area_2 BHK_high,area_2 BHK_low,price-range_2 BHK_high,price-range_2 BHK_low,area_3 BHK_high,area_3 BHK_low,price-range_3 BHK_high,price-range_3 BHK_low,area_4 BHK_high,area_4 BHK_low,price-range_4 BHK_high,price-range_4 BHK_low,area_5 BHK_high,area_5 BHK_low,price-range_5 BHK_high,price-range_5 BHK_low,area_6 BHK_high,area_6 BHK_low,price-range_6 BHK_high,price-range_6 BHK_low,area_Land_high,area_Land_low,price-range_Land_high,price-range_Land_low,building_type_1 BHK_Service Apartment,building_type_2 BHK_Independent Floor,building_type_2 BHK_Service Apartment,building_type_3 BHK_Independent Floor,building_type_3 BHK_Service Apartment,building_type_3 BHK_Villa,building_type_4 BHK_Independent Floor,building_type_4 BHK_Villa,building_type_5 BHK_Independent Floor,building_type_5 BHK_Villa,building_type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1370.0,1370.0,2.40,2.00,2050.0,1850.0,3.59,2.25,2600.0,2600.0,4.56,3.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0
M3M Crown,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,2170.0,1605.0,3.03,2.20,2670.0,2248.0,3.73,3.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0
Adani Brahma Samsara Vilasa,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,3150.0,1800.0,15.75,2.43,4500.0,2750.0,22.50,3.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4329.0,500.0,41.13,2.05,0,0,0,1,0,0,1,0,0,0,0
Sobha City,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1692.0,1381.0,3.21,1.55,2343.0,1711.0,4.79,1.76,2963.0,2423.0,6.06,2.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0
Signature Global City 93,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1118.0,981.0,1.06,93.01,1530.0,1235.0,1.45,1.12,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,964.0,964.0,1.35,1.35,1127.0,1127.0,1.55,1.55,1562.0,1562.0,2.15,2.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0
Pyramid Urban Homes 2,0.0,0.0,0.0,0.0,398.0,335.0,27.86,23.45,625.0,500.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0
Satya The Hermitage,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1450.0,1450.0,80.00,80.00,1991.0,1991.0,1.00,1.00,4711.0,2639.0,2.14,1.20,4731.0,4731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0


In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df), columns = ohe_df.columns, index = ohe_df.index)

In [65]:
ohe_df_normalized.sample(5)

,area_1 RK_high,area_1 RK_low,price-range_1 RK_high,price-range_1 RK_low,area_1 BHK_high,area_1 BHK_low,price-range_1 BHK_high,price-range_1 BHK_low,area_2 BHK_high,area_2 BHK_low,price-range_2 BHK_high,price-range_2 BHK_low,area_3 BHK_high,area_3 BHK_low,price-range_3 BHK_high,price-range_3 BHK_low,area_4 BHK_high,area_4 BHK_low,price-range_4 BHK_high,price-range_4 BHK_low,area_5 BHK_high,area_5 BHK_low,price-range_5 BHK_high,price-range_5 BHK_low,area_6 BHK_high,area_6 BHK_low,price-range_6 BHK_high,price-range_6 BHK_low,area_Land_high,area_Land_low,price-range_Land_high,price-range_Land_low,building_type_1 BHK_Service Apartment,building_type_2 BHK_Independent Floor,building_type_2 BHK_Service Apartment,building_type_3 BHK_Independent Floor,building_type_3 BHK_Service Apartment,building_type_3 BHK_Villa,building_type_4 BHK_Independent Floor,building_type_4 BHK_Villa,building_type_5 BHK_Independent Floor,building_type_5 BHK_Villa,building_type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Smart World Orchard,-0.10253,-0.105157,-0.082725,-0.090521,-0.169584,-0.252266,-0.134584,-0.1284,0.852209,0.806272,-0.308366,-0.400560,0.158606,0.181357,-0.089392,-0.210192,-0.943016,-1.022842,-0.544815,-0.545232,-0.460463,-0.468954,-0.26925,-0.290991,-0.118934,-0.125582,-0.073387,-0.077649,-0.371421,-0.447044,-0.245503,-0.212932,-0.111111,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Capital,-0.10253,-0.105157,-0.082725,-0.090521,-0.169584,-0.252266,-0.134584,-0.1284,0.964137,1.115329,-0.320911,-0.412276,0.073702,0.340970,-0.157829,-0.240933,0.056136,0.262070,0.053827,0.316336,-0.460463,-0.468954,-0.26925,-0.290991,-0.118934,-0.125582,-0.073387,-0.077649,-0.371421,-0.447044,-0.245503,-0.212932,-0.111111,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Emaar The Palm Springs,-0.10253,-0.105157,-0.082725,-0.090521,-0.169584,-0.252266,-0.134584,-0.1284,-0.896660,-0.893541,-0.382065,-0.455791,1.750547,1.309288,0.517980,-0.026259,1.711469,1.259362,1.664808,1.633566,1.375948,1.774371,-0.26925,-0.290991,-0.118934,-0.125582,-0.073387,-0.077649,-0.371421,-0.447044,-0.245503,-0.212932,-0.111111,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,4.622081,-0.063888
Mapsko The Icon 79,-0.10253,-0.105157,-0.082725,-0.090521,-0.169584,-0.252266,-0.134584,-0.1284,-0.896660,-0.893541,-0.382065,-0.455791,-1.369658,-1.414772,-0.378108,-0.325471,0.611912,1.165573,-0.059736,0.237774,-0.460463,-0.468954,-0.26925,-0.290991,-0.118934,-0.125582,-0.073387,-0.077649,-0.371421,-0.447044,-0.245503,-0.212932,-0.111111,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
BPTP Pedestal,-0.10253,-0.105157,-0.082725,-0.090521,-0.169584,-0.252266,-0.134584,-0.1284,-0.018028,0.076898,-0.382065,-0.455791,-0.566471,-0.408147,-0.378108,-0.325471,-0.389462,-0.243766,-0.544815,-0.545232,-0.460463,-0.468954,-0.26925,-0.290991,-0.118934,-0.125582,-0.073387,-0.077649,-0.371421,-0.447044,-0.245503,-0.212932,-0.111111,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [66]:
ohe_df_normalized.shape

(246, 43)

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(ohe_df_normalized, ohe_df_normalized)

In [68]:
cosine_sim2.shape

(246, 246)

In [69]:
def recommed_properties_with_scores(property_name, top_n = 247):
  sim_scores = list(enumerate(cosine_sim2[ohe_df_normalized.index.get_loc(property_name)]))
  sorted_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
  top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
  top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
  top_properties = ohe_df_normalized.index[top_indices].tolist()
  return pd.DataFrame({'PropertyName': top_properties, 'SimilarityScore': top_scores})

In [70]:
recommed_properties_with_scores('M3M Golf Hills')

,PropertyName,SimilarityScore
0,Smartworld One DXP,0.986377
1,Sobha City,0.976233
2,Unitech Escape,0.967197
3,BPTP Terra,0.964302
4,M3M Capital,0.963609
...,...,...
240,BPTP Fortuna,-0.406928
241,Orris Woodview Residencies,-0.407119
242,Vatika India Next Plots,-0.408289
243,Anant Raj Estate Plots,-0.410724


In [71]:
recommed_properties_with_scores('Sobha City')

,PropertyName,SimilarityScore
0,M3M Golf Hills,0.976233
1,Smartworld One DXP,0.970015
2,Ireo Skyon,0.964428
3,AIPL The Peaceful Homes,0.959651
4,Unitech Escape,0.958590
...,...,...
240,Anant Raj Ashok Estate,-0.463050
241,Satya Merano Greens,-0.463148
242,ROF Normanton Park,-0.464583
243,India Rashtra,-0.465224


In [72]:
# 3rd recommender system

In [73]:
df[['PropertyName', 'LocationAdvantages']]['LocationAdvantages'].sample().values[0]

"{'Dwarka Expressway': '1.2 Km', 'S N International School': '1.2 Km', 'InfinityS Badminton Academy': '3.1 Km', 'Gurgaon Railway Station': '3.3 Km', 'Gurgaon Dreamz Mall': '4.6 Km', 'Metro Hospital, Palam Vihar': '6.4 Km', 'Delhi Jaipur Expressway': '7.7 Km', 'HUDA Market, Sector 14': '7.9 Km', 'F9 Go Karting Gurgaon': '8.4 Km', 'The NorthCap University': '8.8 Km', 'infinity Business Park': '9.4 Km', 'Hero Honda Chowk': '9.4 Km', 'SkyJumper Trampoline Park Gurgaon': '10.1 Km', 'Huda Metro Station': '11.6 Km', 'Indira Gandhi International Airport': '21.7 Km'}"

In [74]:
df['LocationAdvantages'].shape

(246,)

In [75]:
# find feature set

In [76]:
!pip install pandas fuzzywuzzy python-Levenshtein


In [77]:
import pandas as pd
import ast
from fuzzywuzzy import fuzz, process

# Step 1: Parse LocationAdvantages column from string to dict
def parse_dict(row):
    try:
        return ast.literal_eval(row)
    except:
        return {}

df["LocationAdvantages"] = df["LocationAdvantages"].astype(str).apply(parse_dict)

# Step 2: Collect all keys
all_keys = []
for row in df["LocationAdvantages"]:
    all_keys.extend(row.keys())

# Step 3: Fuzzy group similar keys
unique_keys = list(set(all_keys))
standardized_keys = {}

# Choose a threshold (70–85 is typical for location names)
THRESHOLD = 85
used_keys = []

for key in unique_keys:
    if key in used_keys:
        continue
    matches = process.extract(key, unique_keys, scorer=fuzz.token_sort_ratio)
    group = [m[0] for m in matches if m[1] >= THRESHOLD]
    for g in group:
        standardized_keys[g] = key  # Assign group member to main key
    used_keys.extend(group)

# Step 4: Replace fuzzy keys with standard ones
def normalize_keys(d):
    new_dict = {}
    for k, v in d.items():
        std_key = standardized_keys.get(k, k)
        if std_key in new_dict:
            # Keep the closer distance (lower KM)
            try:
                old_val = float(new_dict[std_key].split()[0])
                new_val = float(v.split()[0])
                if new_val < old_val:
                    new_dict[std_key] = v
            except:
                pass
        else:
            new_dict[std_key] = v
    return new_dict


In [78]:
cleaned_locations = df["LocationAdvantages"].apply(normalize_keys)

In [79]:
cleaned_locations.sample(2)

,LocationAdvantages
20,"{'Mavens Inn': '350 M', 'Sanar International Hospital': '500 M', 'Central Plaza Mall': '1 Km', 'Sector 53/54 Metro Station': '1.3 Km', 'Gurugram University': '1.6 Km', 'The Banyan Tree World School': '2 Km', 'The Big Tree Cafe': '2.7 Km', 'DLF Golf and Country Club': '4.6 Km', 'DoubleTree by Hilton Hotel Gurgaon': '1.6 Km'}"
60,"{'SCC Drive-In Cinema': '0.8 Km', 'Heritage Intl Xperiential School': '1.4 Km', 'Paras Trinity Shopping Mall': '1.8 Km', 'Swastik Multispeciality Hospital': '2 Km', 'AIPL Business Co Working Space': '2.4 Km', 'Lemon Tree Hotel Sector 60': '2.4 Km', 'Golf Course Extension Road': '2.9 Km', 'Gurugram University': '3.9 Km', 'SkyJumper Trampoline Park Gurgaon': '7.9 Km', 'DLF Golf and Country Club': '8.7 Km', 'Indira Gandhi International Airport': '20.6 Km'}"


In [80]:
# go to each location and fi

In [81]:
type(cleaned_locations.iloc[0])

dict

In [82]:
all_keys = set()
for locs in cleaned_locations:
      all_keys.update(locs.keys())

all_keys = sorted(list(all_keys))

In [83]:
all_keys

['AIIMS',
 'AIIMS Jhajjar',
 'AIPL Business Centre',
 'AIPL Business Club',
 'AIPL Business Club Sector 62',
 'AIPL Business Co Working Space',
 'AIPL Business Tower',
 'AIPL Joy Street Mall',
 'APJ Abdul Kalam Park',
 'ASF Insignia SEZ',
 'Aapno Ghar',
 'Aarvy Healthcare',
 'Aarvy Healthcare Hospital',
 'Aarvy Healthcare Super Speciality',
 'Aatish Hospital',
 'Accenture DDC5',
 'Adarsh public school,Garhi Harsaru',
 'Agri Business Management Collage',
 'Airia Mall',
 'Airia Mall Sector 68',
 'Airport',
 'Ajit Stadium Dhanwapur',
 'Alfaa Health Care Hospital',
 'Alpine Convent School',
 'Alpine Hospital',
 'Alpine School',
 'Altrade Business Centre',
 'Aman Hospital & Surgical Centre',
 'Ambience Mall',
 'Ambience Public School',
 'American Express',
 'Amity',
 'Amity University',
 'Amma Hospital',
 'Anand Multispeciality Hospital',
 'Anand Preschool',
 'Ananta Hospital',
 'Ansal Plaza',
 'Anya Gurgaon',
 'Ap Sports cricket ground',
 'Apex Plus Hospital',
 'Apollo',
 'Apollo Pharmacy'

In [84]:
len(all_keys)

871

In [85]:
feature_set = list(all_keys)

In [86]:
cleaned_locations.index = df['PropertyName']

In [87]:
def distance_to_meters(distance_str) :
  distance_str = distance_str.lower()
  try :
    if 'km' in distance_str :
      return float(distance_str.split()[0]) * 1000
    elif 'meter' in distance_str :
      return float(distance_str.split()[0])
    else :
      return None
  except:
    return None

In [88]:
def extract_location (index, new_df) :
  output = []
  row = cleaned_locations.loc[index]
  for key in feature_set :
    if key in row :
      output.append(distance_to_meters(row[key]))
    else :
      output.append(None)
  new_df.loc[index] = output

In [89]:
new_df = pd.DataFrame(columns=feature_set, index=cleaned_locations.index)

In [90]:
len(feature_set)

871

In [91]:
new_df.shape

(246, 871)

In [92]:
for i in cleaned_locations.index:
  extract_location(i, new_df)

In [93]:
new_df.shape

(246, 871)

In [94]:
new_df.sample(5)

,AIIMS,AIIMS Jhajjar,AIPL Business Centre,AIPL Business Club,AIPL Business Club Sector 62,AIPL Business Co Working Space,AIPL Business Tower,AIPL Joy Street Mall,APJ Abdul Kalam Park,ASF Insignia SEZ,Aapno Ghar,Aarvy Healthcare,Aarvy Healthcare Hospital,Aarvy Healthcare Super Speciality,Aatish Hospital,Accenture DDC5,"Adarsh public school,Garhi Harsaru",Agri Business Management Collage,Airia Mall,Airia Mall Sector 68,Airport,Ajit Stadium Dhanwapur,Alfaa Health Care Hospital,Alpine Convent School,Alpine Hospital,Alpine School,Altrade Business Centre,Aman Hospital & Surgical Centre,Ambience Mall,Ambience Public School,American Express,Amity,Amity University,Amma Hospital,Anand Multispeciality Hospital,Anand Preschool,Ananta Hospital,Ansal Plaza,Anya Gurgaon,Ap Sports cricket ground,Apex Plus Hospital,Apollo,Apollo Pharmacy,Approved Sector 37 Mero Station,Appu Ghar,Appu Ghar Water Park,Aradhya Cricket Club Gurgaon,Aravali Adventure Hill,Aravalli Hill View Point,Aravalli Hills,Arc Multi Speciality,Ardee Mall,Artemis,Artemis Hospital Gurgaon,Artimis hospital,Aryan Hospital,Ascendas OneHub Gurgaon,Ascendas OneHub Gurgaon Business Park,Ashiana Anmol Kid Centric Homes,Athena,Au Grand Air,Axis Bank,Axis Bank ATM,"Axis Bank, Sohna Rd",BM College of Technology & Mgmt,BML Munjal University (BMU),BOB ATM,BSF Golf Course,Baba Kanala Chowk,Badsa AMS Hospital,"Badshahpur Sohna Rd Hwy, Haryana","Badshahpur Sohna Rd Hwy, Malibu Town","Badshahpur Sohna Rd Hwy, Raghav Vatika","Badshahpur Sohna Rd Hwy, Rajoria Ngr","Badshahpur Sohna Rd Hwy,Sector 48",Badshapur Sohna Highway,Baghera University,Bajghera Road,Bal Bharati Public School,Balaji Hospital,Banjara Market Gurugram,Bank Of Baroda ATM,Basai Dhancourt Railway Station,Basai Dhankot,Basai Enclave Park,Basai Metro Station,Basai Road,Basant Valley Global School,Best IVF Centre,Bestech Business Tower,Bestech Central Square Mall,Bharat Petrol Pump,Bharat Petroleum Petrol Pump,Bharat Petroleum Retail Outlet,Bharat Petroleum Shree Shyam Filling,Bharat Ram Global School,Bharat Singh fuel company,Bharti International Convent School,Bhondsi Nature Park,BigBazaar,Bijwasan Railway Station,Biryani Shah,Blue Bells Public School,Broadways International School,CBR Cricket Ground,CK Birla Hospital,"CK Birla Hospital, Gurgaon",CNG Petrol Pump,Cambridge College Of Education,Cambridge Montessori,Cambridge Montessori Preschool,Cambridge Pre-School,Canara Bank - Nawada Fatehpur,Canara Bank ATM,Canara Bank New Palam Vihar,"Candor TechSpace, Sector 48",Candor Techspace,Capital Business Park,Capital Cyberscape,Captain Chandan Lal Marg,Central Bank Of India Sohna Rd,Central Park Flower Valley,Central Park II Road,Central Park Resorts,"Central Park, Sohna Rd",Central Peripheral Road,Central Plaza Mall,Centrum Plaza,Chauma Road,Cherub's Cradle,Children Park,Chirag Hospital,Choice pharmacy,Citibank ATM,City Hospital,City Square,Civil Hospital,Cloudnine Hospital Sector 47,CoNexus.Life B35,Colonel's Central Academy,Conscient One,Conscient One Mall,Cool Deck Coffee,Country Inn,Country Inn & Suites By Radisson,Creative Tennis Academy,Cricket Academy,Cyber ​​Park,DLC Cricket Ground,DLF Corporate Greens,DLF Corporate Park,DLF Cyber City,DLF Golf and Country Club,DLF Grand Mall,DLF Linear Park,DLF Site central office,DLF World Tech Park,DLF5 Summit Plaza,DPG Degree College,DPG Institute of Technology,DPGITM Engineering College Sector 34,DPS,DPS International Edge,DPS International School,DPS Manesar,DPS Sector 103,DPSG Palam Vihar,DPSG Palam Vihar Gurugram,DSD College,Damdama Lake Rd,Damdama More,Daultabad Stadium,Daultabad Village Park,De Adventure Amusement Park,De Adventure Park,Deerika HyperMart,Delh-Ajmer Expy,Delhi,Delhi - Jaipur Expressway,Delhi Ajmer Expressway,Delhi Gurgaon Expressway,Delhi Public School,Delhi Public School Gurugram Sector 67A,"Delhi Public School, Sector 103",Delhi-Mumbai Expressway,Dhanwapur Road,Dharampur Main Road,Diamond Public School,Discount Department Store,Dishoom Cinemas,Domino's Pizza,Double I

In [95]:
cleaned_locations.loc['Ireo Victory Valley']

{'AIPL Joy Street Mall': '1.1 Km',
 'Alpine Convent School': '1.8 Km',
 'Golf Course Extension Road': '3.3 Km',
 'CK Birla Hospital': '4.5 Km',
 'PVR Drive In Theatre': '4.8 Km',
 'Radisson Hotel Gurugram Sohna Road': '5 Km',
 'SkyJumper Trampoline Park Gurgaon': '5.6 Km',
 'DPG Institute of Technology': '8.3 Km',
 'iON Digital Zone (Gurugram)': '8.3 Km',
 'HUDA Mini Golf Course': '9.2 Km',
 'Indira Gandhi International Airport': '24 Km'}

In [96]:
# fill the missing values with a big value
# indicating that index (i) and and column (j) are very far from each other
new_df.fillna(60000, inplace = True)

<ipython-input-96-8db8ef4ad0df>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_df.fillna(60000, inplace = True)


In [97]:
new_df.isnull().sum()

,0
AIIMS,0
AIIMS Jhajjar,0
AIPL Business Centre,0
AIPL Business Club,0
AIPL Business Club Sector 62,0
...,...
iGrow Montessori,0
iGrow Montessori Play School,0
iON Digital Zone (Gurugram),0
infinity Business Park,0


In [98]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
new_df_normalized = pd.DataFrame(scaler.fit_transform(new_df), columns = new_df.columns, index = new_df.index)

In [99]:
cosine_sim3 = cosine_similarity(new_df_normalized)

In [100]:
cosine_sim3.shape

(246, 246)

In [101]:
def recommed_properties_with_scores3(property_name, top_n = 247):
  cosine_sim_matrix = cosine_sim3
  sim_scores = list(enumerate(cosine_sim_matrix[new_df_normalized.index.get_loc(property_name)]))
  sorted_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
  top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
  top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
  top_properties = new_df_normalized.index[top_indices].tolist()
  return pd.DataFrame({'PropertyName': top_properties, 'SimilarityScore': top_scores})

In [102]:
recommed_properties_with_scores('Ireo Victory Valley')

,PropertyName,SimilarityScore
0,M3M Golfestate,0.964670
1,Ambience Creacions,0.945343
2,SS The Leaf,0.905778
3,DLF The Crest,0.889412
4,Pioneer Araya,0.806337
...,...,...
240,Shree Vardhman City,-0.700240
241,Rishali Luxe Residency 112,-0.701173
242,Satya Merano Greens,-0.703154
243,Pyramid Spring Valley,-0.705252


In [103]:
cosine_sim2

array([[ 1.        ,  0.15016605, -0.1345584 , ...,  0.18151971,
         0.42943286,  0.59335441],
       [ 0.15016605,  1.        ,  0.008117  , ..., -0.34916688,
         0.05623759, -0.07830163],
       [-0.1345584 ,  0.008117  ,  1.        , ..., -0.17819172,
        -0.36106425, -0.33112597],
       ...,
       [ 0.18151971, -0.34916688, -0.17819172, ...,  1.        ,
        -0.16610953, -0.0281491 ],
       [ 0.42943286,  0.05623759, -0.36106425, ..., -0.16610953,
         1.        ,  0.94733391],
       [ 0.59335441, -0.07830163, -0.33112597, ..., -0.0281491 ,
         0.94733391,  1.        ]], shape=(246, 246))

In [104]:
cosine_sim1

array([[1.        , 0.01095159, 0.        , ..., 0.01183329, 0.08656385,
        0.0110727 ],
       [0.01095159, 1.        , 0.01982121, ..., 0.11904241, 0.01555534,
        0.00963852],
       [0.        , 0.01982121, 1.        , ..., 0.07020502, 0.03820314,
        0.01962826],
       ...,
       [0.01183329, 0.11904241, 0.07020502, ..., 1.        , 0.09825738,
        0.03255851],
       [0.08656385, 0.01555534, 0.03820314, ..., 0.09825738, 1.        ,
        0.06257614],
       [0.0110727 , 0.00963852, 0.01962826, ..., 0.03255851, 0.06257614,
        1.        ]], shape=(246, 246))

In [105]:
cosine_sim3

array([[ 1.        ,  0.07864331, -0.01342029, ...,  0.00292218,
        -0.01048526, -0.00193488],
       [ 0.07864331,  1.        , -0.01037644, ..., -0.00638181,
        -0.0187956 , -0.01670982],
       [-0.01342029, -0.01037644,  1.        , ..., -0.02221424,
        -0.01953766, -0.01818547],
       ...,
       [ 0.00292218, -0.00638181, -0.02221424, ...,  1.        ,
         0.12373199, -0.034933  ],
       [-0.01048526, -0.0187956 , -0.01953766, ...,  0.12373199,
         1.        , -0.02926031],
       [-0.00193488, -0.01670982, -0.01818547, ..., -0.034933  ,
        -0.02926031,  1.        ]], shape=(246, 246))

In [106]:
cleaned_locations.sample(10)

,LocationAdvantages
PropertyName,
Emaar MGF Emerald Floors Premier,"{'WorldMark Gurgaon': '400 m', 'Sohna Road': '3.2 Km', 'Lemon Tree Hotel,': '3.9 Km', 'CK Birla Hospital': '5.2 Km', 'Golf Course Ext Road': '5.6 Km', 'IG International Airport': '23.4 Km'}"
DLF Park Place,"{'Sector 53/54 Metro Station': '1.7 km', 'Health Care Pharmacy': '2.6 km', 'Paras Hospitals, Gurgaon': '3.3 km', 'Sector 54 Chowk': '3.9 km', 'Gurugram University': '3.9 km', 'EuroKids Preschool Suncity': '4.6 km', 'YES Bank ATM': '4.7 km', 'Sushant University': '4.8 km', 'Suncity School Gurgaon': '5.3km', 'Suncity Shopping Complex': '5.6 km', 'Apollo Pharmacy': '5.6 km', 'Vallores Pre School': '5.8km', 'W Pratiksha Hospital': '6.3 km', 'MKD Hospital': '7.1 km', 'Haryana City Gas': '8 km'}"
DLF The Grove,"{'Park Dr, DLF Phase 5': '41 Meter', 'Central Plaza Mall': '1.8 KM', 'Sector 53/54 Metro Station': '2.1 KM', 'Sushant University': '2.5 KM', 'Unicosmos School': '2.8 KM', 'Paras Hospitals': '4.2 KM', 'NH 248A': '10 KM', 'Indira Gandhi International Airport': '16.9 KM'}"
BPTP Amstoria,"{'Dwaraka Expressway': '1.3Km', 'Early Basket Grocery shop': '2Km', 'Aradhya Cricket Club Gurgaon': '2.7Km', 'Imperial Heritage School': '2.9Km', 'Daultabad Village Park': '3.6Km', 'Skylark Cricket Academy': '3.8Km', 'Yashroop Hospital': '4.7Km', 'Gurgaon Old Railway Station': '7.8Km', 'Domino's Pizza': '8Km', 'Conscient One Mall': '8.6Km', 'IGI Airport': '24.8Km'}"
Godrej Aria,"{'Mount Olympus School, Sec 79': '170 Meter', 'Savoy Suites, Manesar': '6.1 Km', 'Delhi - Jaipur Expressway': '4.2 Km', 'Nakhrola Stadium': '5.7 Km', 'Sapphire 83 Mall': '6 Km', 'Miracles Apollo Hospital': '5.5 Km', 'Singhania University, Manesar': '10.4 Km', 'Capital Business Park': '12 Km', 'Manesar Golf Course': '9.7 Km', 'Garhi Harsaru Junction': '11.8 Km', 'Indira Gandhi International Airport': '31.2 Km'}"
Pyramid Urban Homes 2,"{'Aarvy Healthcare Super Speciality': '1.8 KM', 'St. Xavier's High School': '2 KM', 'Minda Industries Corporate Office': '3.1 KM', 'Sapphire 83 Mall': '3.2 KM', 'Rampura Flyover, Naurangpur Rd': '3.7 KM', 'Nakhrola Stadium': '3.8 KM', 'Manesar toll plaza - Kherki Daula': '6.1 KM', 'Imt Manesar, Gurugram': '7.7 KM', 'Holiday Inn': '3 km', 'Indira Gandhi International Airport': '29.4 Km'}"
Trump Tower,"{'WorldMark Gurgaon': '300 M', 'Global Ways School': '3.5 Km', 'Paras Hospitals': '4.4 Km', 'Gurugram University': '5.2 Km', 'Radisson Hotel': '5.3 Km', 'NH 248A': '6.3 Km', 'Sector 55-56 Metro Station': '6.5 Km', 'Garhi Harsaru Junction': '17.8 Km', 'Indira Gandhi International Airport': '25.2 Km'}"
M3M Soulitude,"{'Sector 86 Road': '950 Meter', 'Genesis Hospital': '1.5 KM', 'Dwarka Expy': '2.5 KM', 'Euro International School': '2.6 KM', 'Vatika Town Square': '4.1 KM', 'Garhi Harsaru Junction': '4.6 KM', 'Delh-Ajmer Expy': '5.8 KM', 'Gurugram University': '15.7 KM', 'Indira Gandhi International Airport': '28 KM'}"
BPTP Fortuna,"{'Southern Peripheral Rd, Dhani': '2.6 KM', 'Euro International School': '5.2 KM', 'Omaxe City Centre Mall': '5.5 KM', 'Badshahpur Sohna Rd Hwy, Malibu Town': '7.5 KM', 'Paras Hospitals': '7.6 KM', 'Gurugram University': '8.8 KM', 'Sector 53/54 Metro Station': '10.7 KM', 'Indira Gandhi International Airport': '25.4 KM'}"


In [107]:
new_df.head(10)

AIIMS  AIIMS Jhajjar  AIPL Business Centre  \
PropertyName                                                              
Smartworld One DXP           60000        60000.0               60000.0   
M3M Crown                    60000        60000.0               60000.0   
Adani Brahma Samsara Vilasa  60000        60000.0               60000.0   
Sobha City                   60000        60000.0               60000.0   
Signature Global City 93     60000        60000.0               60000.0   
Whiteland The Aspen          60000        60000.0               60000.0   
Bestech Altura               60000        60000.0               60000.0   
Elan The Presidential        60000        60000.0               60000.0   
Signature Global City 92     60000        60000.0               60000.0   
Emaar Digihomes              60000        60000.0               60000.0   

                             AIPL Business Club  AIPL Business Club Sector 62  \
PropertyName                                                                    
Smartworld One DXP                      60000.0                       60000.0   
M3M Crown                               60000.0                       60000.0   
Adani Brahma Samsara Vilasa             60000.0                        2700.0   
Sobha City                              60000.0                       60000.0   
Signature Global City 93                60000.0                       60000.0   
Whiteland The Aspen                     60000.0                       60000.0   
Bestech Altura                          60000.0                       60000.0   
Elan The Presidential                   60000.0                       60000.0   
Signature Global City 92                60000.0                       60000.0   
Emaar Digihomes                         60000.0                       60000.0   

                             AIPL Business Co Working Space  \
PropertyName                                                  
Smartworld One DXP                                  60000.0   
M3M Crown                                           60000.0   
Adani Brahma Samsara Vilasa                         60000.0   
Sobha City                                          60000.0   
Signature Global City 93                            60000.0   
Whiteland The Aspen                                 60000.0   
Bestech Altura                                      60000.0   
Elan The Presidential                               60000.0   
Signature Global City 92                            60000.0   
Emaar Digihomes                                     60000.0   

                             AIPL Business Tower  AIPL Joy Street Mall  \
PropertyName                                                             
Smartworld One DXP                       60000.0               60000.0   
M3M Crown                                60000.0               60000.0   
Adani Brahma Samsara Vilasa              60000.0               60000.0   
Sobha City                               60000.0               60000.0   
Signature Global City 93                 60000.0               60000.0   
Whiteland The Aspen                      60000.0               60000.0   
Bestech Altura                           60000.0               60000.0   
Elan The Presidential                    60000.0               60000.0   
Signature Global City 92                 60000.0               60000.0   
Emaar Digihomes                            160.0               60000.0   

                             APJ Abdul Kalam Park  ASF Insignia SEZ  \
PropertyName                                                          
Smartworld One DXP                          60000           60000.0   
M3M Crown                                   60000           60000.0   
Adani Brahma Samsara Vilasa                 60000           60000.0   
Sobha City                                  60000           60000.0   
Signature Global City 93                    60000           60000.0   
Whiteland Th

In [108]:
np.__version__

'2.2.4'

In [109]:
new_df.to_csv('location_df.csv', index=False)

In [112]:
np.save('cosine_sim1.npy', cosine_sim1)

In [114]:
np.save('cosine_sim2.npy', cosine_sim2)
np.save('cosine_sim3.npy', cosine_sim3)